### Json Parsing and Processing

In [ ]:
import json
import os

os.makedirs("data/json_files", exist_ok=True)

In [2]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [3]:
json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'Data Scientist',
   'skills': ['Python', 'Machine Learning', 'SQL'],
   'projects': [{'name': 'ML Model', 'status': 'In Progress'},
    {'name': 'Analytics Dashboard', 'status': 'Planning'}]}],
 'departments': {'engineering': {'head': 'Mike Johnson',
   'budget': 1000000,
   'team_size': 25},
  'data_science': {'head': 'Sarah Williams',
   'budget': 750000,
   'team_size': 15}}}

In [4]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [5]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

### JSON Processing Strategies

In [8]:
from langchain_community.document_loaders import JSONLoader
import json

#Method 1: JsonLoader with js_schema
print('JSONLoader - Extract specific fields')

#Extract employee information
employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',
    jq_schema='.employees[]', #jq qeury to extract each employee
    text_content=False #Get Full JSON objects
)

employee_docs = employee_loader.load()
print(f"Loaded {len(employee_docs)} employee documents")
print(f"First employee {employee_docs[0].page_content[:200]}...")
print(employee_docs)

JSONLoader - Extract specific fields
Loaded 2 employee documents
First employee {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...
[Document(metadata={'source': 'E:\\workspace\\ultimate_rag_bootcamp\\0-DataIngestParsing\\data\\json_files\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'E:\\workspace\\ultimate_rag_bootcamp\\0-DataIngestParsing\\data\\json_files\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"name": "ML Model", "status": "In Progress"}, {"name": "Analyt

In [22]:
from langchain_core.documents import Document
from typing import List

print("\n Custom JSON Processing")

def process_json_intelligently(filepath:str)-> List[Document]:
    """Process JSON with intelligent flattening and context preservation"""
    with open(filepath, 'r') as f:
        data = json.load(f)

    documents = []

    #Strategy 1: Create documents for each employee with full context 

    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name : {emp['name']}
        Role : {emp['role']}
        Skills:{emp['skills']}
        
        Projects:"""
        for proj in emp.get('projects', []):
            content += f"\n - {proj['name']} (Status: {proj['status']})"
        
        doc = Document(
            page_content=content, 
            metedata = {
                'source':filepath,
                'data_type':'employee_profile', 
                'employee_id':emp['id'], 
                'employee_name':emp['name'], 
                'role':emp['role']
            }
        )
        documents.append(doc)

    return documents



 Custom JSON Processing


In [27]:
json_data = process_json_intelligently('data/json_files/company_data.json')
print(json_data[0].page_content)

Employee Profile:
        Name : John Doe
        Role : Software Engineer
        Skills:['Python', 'JavaScript', 'React']

        Projects:
 - RAG System (Status: In Progress)
 - Data Pipeline (Status: Completed)
